In [ ]:
import pandas as pd
import pyreadr  #pip install pyreadr -q -- apply this if pyreadr is not giving Error
import matplotlib.pyplot as plt
import numpy as np
import time
import sys
from numpy import array
from numpy.linalg import pinv
from sklearn.preprocessing import MinMaxScaler
from sklearn import linear_model
from sklearn.metrics import r2_score

In [ ]:
result = pyreadr.read_r(r"C:\Users\alivi\Desktop\Ali Vijdaan\Jupyter Notebooks\ES-205 Project\movies.RData") # Read .rdata file
# Extract the dataframe from the result
df = result["movies"]  # Assuming the dataframe is the first object in the RData file

# Convert the dataframe to a CSV file
csv_file_path = r"C:\Users\alivi\Desktop\Ali Vijdaan\Jupyter Notebooks\ES-205 Project\movies.csv"
df.to_csv(csv_file_path, index=False)  # Save as CSV without the index

# Display the first few rows of the CSV file
df = pd.read_csv(csv_file_path)
df.head()

In [ ]:
df.head()

In [ ]:
#Showing data with Selected Colomns
selected_columns = ['title', 'mpaa_rating','genre', 'critics_score', 'audience_score']
s_df = df[selected_columns] #s_df indicates selected dataframe
s_df.head(15)

In [ ]:
x = s_df['audience_score']
y = s_df['critics_score']

plt.figure(figsize=(10, 5)) #(width, height)

plt.scatter(x, y)

plt.title('Scatter Plot of Critic Score vs Audience Score')
plt.xlabel('Audience Score')
plt.ylabel('Critic Score')

plt.show()

Gradient Descent

In [ ]:
def gradient_descent(x, y):
  
  m_curr = 0    #Initialzing our predictions for gradeint
  b_curr = 0    #Initialzing our predictions for y-intercept
  iterations = 1000
  n = len(x)
  learning_rate = 0.000001

  res_m = []
  res_b = []
  res_mse = []

  for i in range(iterations):
    y_hat = m_curr * x + b_curr  #Equation for line of predicted values

    mse = (1/n) * sum((y - y_hat)**2)
    res_mse.append(mse)

    Dm = (-2/n) * sum(x*(y - y_hat))  #Partial Derivative wrt m
    Db = (-2/n) * sum(y - y_hat)  #Partial Derivative wrt y-intercept

    #Updating m and y-intercept
    m_curr = m_curr - (learning_rate * Dm)
    res_m.append(m_curr)
    b_curr = b_curr - (learning_rate * Db)
    res_b.append(b_curr)
  
  #Printing Results
  print(f"Gradient: {m_curr}")
  print(f"Intercepts: {b_curr}")
  print(f"Mean Squared Error: {mse}")


  return res_m, res_b, res_mse, m_curr, b_curr

In [ ]:
m, c, mse, grad, y_int = gradient_descent(x, y)

In [ ]:
plt.scatter(c, mse, s = 10)

plt.title('MSE vs Intercept')
plt.xlabel('Intercept')
plt.ylabel('MSE')

plt.show()

In [ ]:
plt.scatter(m, mse, s = 10)

plt.title('MSE vs Gradients')
plt.xlabel('Gradients')
plt.ylabel('MSE')


plt.show()

In [ ]:
y_hat = grad * x + y_int

plt.figure(figsize=(10, 5)) #(width, height)
plt.scatter(x, y)
plt.plot(x, y_hat, color = 'red')

plt.title("Linear Regression using Gradient Descent")
plt.xlabel('Audience Scores')
plt.ylabel('Critics Score')

plt.show()

In [ ]:

obs_val = 407.4918887787872
tru_val = 477.1002705961618

acc = 100 - abs((((obs_val - tru_val) / tru_val) * 100))
print(f"Accuracy = {acc}%")